In [29]:
import torch 
import torch.nn as nn

In [63]:
class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, first=None, downsample=None):
        super(Bottleneck, self).__init__()
        self.expansion = 4
        self.downsample = downsample
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU(inplace=True)
           
    def forward(self, x):
        identity = x.copy()
        
        x = self.bn1(self.conv1(x))
        x = self.relu(x)
        x = self.bn2(self.conv2(x))
        x = self.relu(x)
        x = self.bn3(self.conv3(x))
        x = self.relu(x)
        
        if self.downsample is not None:
            identity = self.downsample(identity)
            x += identity
        
        return x

In [73]:
class ResNet(nn.Module):
    def __init__(self, in_channels, num_classes, Bottleneck, layers):
        super(ResNet, self).__init__()
        self.in_channels = in_channels
        
        self.conv1 = nn.Conv2d(in_channels, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.in_channels = 64
        
        self.layer1 = self._make_layer(Bottleneck, layers[0], out_channels=64, stride=1)
        self.layer2 = self._make_layer(Bottleneck, layers[1], out_channels=128, stride=2)
        self.layer3 = self._make_layer(Bottleneck, layers[2], out_channels=256, stride=2)
        self.layer4 = self._make_layer(Bottleneck, layers[3], out_channels=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=1)
        self.fc = nn.Linear(in_features=2048, out_features=num_classes, bias=True)
    
    def forward(self, x):
        x = self.bn1(self.conv1(x))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = self.fc(x)
        
        return x
    
    def _make_layer(self, Bottleneck, num_rep, out_channels, stride):
        layers = []
        
        downsample = nn.Sequential(
                nn.Conv2d(in_channels=self.in_channels, out_channels=out_channels * 4, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * 4)
                )
        
        layers.append(Bottleneck(in_channels=self.in_channels, out_channels=out_channels, stride=stride, downsample=downsample))
        self.in_channels = out_channels * 4
        
        for i in range(num_rep-1):
            layers.append(Bottleneck(in_channels=self.in_channels, out_channels=out_channels))
        
        
        return nn.Sequential(*layers)
       

### Written ResNet models using classes

In [74]:
ResNet50 = ResNet(3, num_classes=1000, Bottleneck=Bottleneck, layers=[3, 4, 6, 3])
ResNet101 = ResNet(3, num_classes=1000, Bottleneck=Bottleneck, layers=[3, 4, 23, 3])
ResNet152 = ResNet(3, num_classes=1000, Bottleneck=Bottleneck, layers=[3, 8, 36, 3])

### PyTorch pretrained ready to use ResNet models

In [54]:
model_50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model_101 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model_152 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)

Using cache found in C:\Users\yerda/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\yerda/.cache\torch\hub\pytorch_vision_v0.10.0
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to C:\Users\yerda/.cache\torch\hub\checkpoints\resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

Using cache found in C:\Users\yerda/.cache\torch\hub\pytorch_vision_v0.10.0
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\yerda/.cache\torch\hub\checkpoints\resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

### Let's load the pre-trained ResNet model's state weights to our models 

In [76]:
ResNet50.load_state_dict(model_50.state_dict())

<All keys matched successfully>

In [77]:
ResNet101.load_state_dict(model_101.state_dict())

<All keys matched successfully>

In [78]:
ResNet152.load_state_dict(model_152.state_dict())

<All keys matched successfully>